In [1]:
run Keypoints_RCNN.ipynb

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-rrno3ym1 because the default path (/home/jetbot/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Matplotlib is building the font cache; this may take a moment.


In [2]:
run 2_Calculating_Angles.ipynb

In [3]:
run 3_Reading_RPLidar.ipynb

In [4]:
run 4_Stage_Estimation.ipynb

# Define Environment

In [5]:
class Environment():
    
    '''
    This wrapper works as an RL-Environment for the Jetson Nano.
    Arguments are: 
        model  -> pytorch model trained in Unity.
        angles -> list of angles required to measure
        FOV    -> How wide the camera lens is. Default = 160
    '''
    
    def __init__(self, model, angles, robot, FOV=160):
        self.previous_readings = {x:0 for x in angles} 
        self.angles = angles
        self.model = model
        self.FOV = FOV
        self.robot = robot
        
        
    def calculate_angle_and_phase(self):
        keypoints, image, counts, objects, peaks = execute({'new': camera.value})
        angle, keypoints = calculate_angle(WIDTH, keypoints, self.FOV)
        phase = self.calculate_phase(keypoints)
        return phase, angle
    
    
    def step(tensor):
        turn, move = tensor.cpu()
        
    
    def stop(self):
        self.robot.stop()

        
    def step_forward(self):
        self.robot.forward(0.4)
        time.sleep(0.5)
        self.robot.stop()

        
    def step_backward(self):
        self.robot.backward(0.4)
        time.sleep(0.5)
        self.robot.stop()

        
    def step_left(self):
        self.robot.left(0.3)
        time.sleep(0.5)
        self.robot.stop()

        
    def step_right(self):
        self.robot.right(0.3)
        time.sleep(0.5)
        self.robot.stop()
    
    
    def calculate_phase(self, keypoints):
        phase = estimating_phase(keypoints)
        return phase
    
    
    def read_lidar(self):
        self.previous_readings = read_lidar_wrapper(self.angles,self.previous_readings)
        
    
    def observe(self):
        self.read_lidar()
        phase, angle = self.calculate_angle_and_phase()
        
        if isinstance(angle, dict):
            angles = list(angle.values())
        
        else: angles = [-1]
        
        observation = phase + angle + list(self.previous_readings.values())
        return observation
    
    
    def sample_action(self, observation):
        observation = torch.Tensor(observation).cuda()
        hidden,_ = self.model.network_body(vis_inputs=[0],vec_inputs=[observation])
        distribution = self.model.distribution(hidden)
        action = distribution.sample()
        return action
    
    
    def step(self,action):
        action = action.cpu().detach().numpy()[0]
        
        speed_move, speed_turn = float(abs(action[0])), float(abs(action[1]))
        
        # backward
        if action[0] < 0:
            robot.backward(speed_move)
            time.sleep(0.5)
            robot.stop()
            
        # forward
        else:
            robot.forward(speed_move)
            time.sleep(0.5)
            robot.stop()
            
        time.sleep(0.2)
            
        # turn left
        if action[1] < 0:
            robot.left(speed_turn)
            time.sleep(0.3)
            robot.stop()
            
        # turn right
        else:
            robot.right(speed_turn)
            time.sleep(0.3)
            robot.stop()


# Load model

In [6]:
model = torch.load("MobileRobot-16325.pth")
model.eval()

SeparateActorCritic(
  (network_body): NetworkBody(
    (visual_processors): ModuleList()
    (vector_processors): ModuleList(
      (0): VectorInput(
        (normalizer): Normalizer()
      )
    )
    (linear_encoder): LinearEncoder(
      (seq_layers): Sequential(
        (0): Linear(in_features=22, out_features=256, bias=True)
        (1): Swish()
        (2): Linear(in_features=256, out_features=256, bias=True)
        (3): Swish()
        (4): Linear(in_features=256, out_features=256, bias=True)
        (5): Swish()
      )
    )
  )
  (distribution): GaussianDistribution(
    (mu): Linear(in_features=256, out_features=2, bias=True)
  )
  (critic): ValueNetwork(
    (network_body): NetworkBody(
      (visual_processors): ModuleList()
      (vector_processors): ModuleList(
        (0): VectorInput(
          (normalizer): Normalizer()
        )
      )
      (linear_encoder): LinearEncoder(
        (seq_layers): Sequential(
          (0): Linear(in_features=22, out_features=256, 

# Define the angles to read using LiDAR

In [7]:
angles = [x for x in range(0,180,10)]

# Initialize the robot

In [8]:
robot = Robot()

In [9]:
env = Environment(model, angles, robot)

In [13]:
observation = env.observe()
print(observation)

[0, 1, 0, 53, 0, 0, 0, 0, 0, 0, 295, 0, 0, 903, 0, 479, 609, 793, 619, 589, 0, 219]


In [17]:
action = env.sample_action(observation)
action

tensor([[ 0.2417, -0.6389]], grad_fn=<AddBackward0>)

In [18]:
env.step(action)